# Text Prediction with LSTMs

Adapted from https://www.tensorflow.org/tutorials/text/text_generation

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
# to load data locally
with open('data/shakespeare.txt', 'r') as f:
    text = f.read()

In [ ]:
# to download data
import requests
resp = requests.get('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = resp.content.decode(encoding='utf-8')

In [3]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[5000:5250])

or heart, the arm our soldier,
Our steed the leg, the tongue our trumpeter.
With other muniments and petty helps
In this our fabric, if that they--

MENENIUS:
What then?
'Fore me, this fellow speaks! What then? what then?

First Citizen:
Should by th


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [7]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char in list(char2idx)[:20]:
    print(f'  {repr(char):4}: {char2idx[char]:3},')
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [10]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [11]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [12]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [15]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(''.join(idx2char[item.numpy()]))
    print('********\n')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
********

are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
********

now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
********

ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
********

one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi
********



In [16]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [17]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [18]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [32]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,
                                    batch_input_shape=[BATCH_SIZE, None]))
model.add(GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'))
model.add(Dense(vocab_size))

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [35]:
example_batch_predictions

<tf.Tensor: shape=(64, 100, 65), dtype=float32, numpy=
array([[[ 6.83475425e-03,  2.17568353e-02,  1.71400351e-03, ...,
         -7.00535858e-03,  3.54460347e-03,  1.15146162e-03],
        [ 3.50874127e-03,  2.02313829e-02,  3.04970611e-03, ...,
         -3.04260813e-02,  1.45373726e-02,  2.28959247e-02],
        [ 9.36156488e-04,  1.45831192e-02,  3.22035304e-03, ...,
         -4.45644893e-02,  2.09487937e-02,  3.68625335e-02],
        ...,
        [ 3.73769808e-03,  1.23782391e-02,  3.54287121e-03, ...,
         -2.60768179e-02, -1.91652973e-03,  1.56679600e-02],
        [ 1.33352149e-02,  1.91539489e-02, -6.11232780e-03, ...,
         -7.11251842e-03, -2.11983942e-03,  2.64493767e-02],
        [-2.40197051e-02,  3.27490480e-03,  3.89770372e-03, ...,
         -1.12645398e-03,  1.22057823e-02,  6.83099031e-03]],

       [[ 1.31722949e-02,  3.88889574e-03, -1.55147794e-03, ...,
         -1.21655958e-02,  1.12607190e-02,  3.25414166e-03],
        [-2.93978211e-03, -7.73710571e-03,  1.29

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 256)           394752    
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            16705     
Total params: 428,097
Trainable params: 428,097
Non-trainable params: 0
_________________________________________________________________


In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [38]:
sampled_indices

array([32, 29, 20, 52,  2, 44, 30, 13, 60, 12, 62, 52,  6, 15, 39, 14, 44,
       40, 30, 38, 31, 54, 13, 36, 14, 39, 42, 51, 34, 28, 46, 56, 49, 12,
        1, 26, 54, 43, 55, 38, 22, 48, 63, 58, 10, 35, 54, 28, 61, 23, 50,
       20, 45, 54, 40,  2, 55, 33,  4, 53,  7, 35, 27, 25, 47, 55, 47, 17,
        0, 55,  8,  7,  2,  1, 40,  8, 11, 20, 54, 23, 32, 64, 10, 16, 46,
       13, 47, 42, 55,  1, 55, 63, 48, 46, 36, 56, 48, 30,  0, 19])

In [39]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "hee with hate,\nDrawn tuns of blood out of thy country's breast,\nAnd cannot live but to thy shame, un"

Next Char Predictions: 
 'TQHn!fRAv?xn,CaBfbRZSpAXBadmVPhrk? NpeqZJjyt:WpPwKlHgpb!qU&o-WOMiqiE\nq.-! b.;HpKTz:DhAidq qyjhXrjR\nG'


In [40]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

def loss(labels, logits):
    return sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1753197


In [41]:
model.compile(optimizer='adam', loss=loss)

In [42]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [43]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 39s 227ms/step - loss: 2.6902
Epoch 2/10
172/172 [==============================] - 38s 223ms/step - loss: 2.0858
Epoch 3/10
172/172 [==============================] - 41s 239ms/step - loss: 1.8631
Epoch 4/10
172/172 [==============================] - 40s 231ms/step - loss: 1.7275
Epoch 5/10
172/172 [==============================] - 46s 269ms/step - loss: 1.6395
Epoch 6/10
172/172 [==============================] - 43s 248ms/step - loss: 1.5781
Epoch 7/10
172/172 [==============================] - 40s 234ms/step - loss: 1.5344
Epoch 8/10
172/172 [==============================] - 40s 235ms/step - loss: 1.5008
Epoch 9/10
172/172 [==============================] - 39s 228ms/step - loss: 1.4761
Epoch 10/10
172/172 [==============================] - 39s 227ms/step - loss: 1.4545


In [44]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [45]:
!ls training_checkpoints/

checkpoint                  ckpt_5.data-00000-of-00001
ckpt_1.data-00000-of-00001  ckpt_5.index
ckpt_1.index                ckpt_6.data-00000-of-00001
ckpt_10.data-00000-of-00001 ckpt_6.index
ckpt_10.index               ckpt_7.data-00000-of-00001
ckpt_2.data-00000-of-00001  ckpt_7.index
ckpt_2.index                ckpt_8.data-00000-of-00001
ckpt_3.data-00000-of-00001  ckpt_8.index
ckpt_3.index                ckpt_9.data-00000-of-00001
ckpt_4.data-00000-of-00001  ckpt_9.index
ckpt_4.index


In [47]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [48]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 256)            394752    
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             16705     
Total params: 428,097
Trainable params: 428,097
Non-trainable params: 0
_________________________________________________________________


In [58]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 2

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
  
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
  
        text_generated.append(idx2char[predicted_id])
  
    return (start_string + ''.join(text_generated))

In [59]:
print(generate_text(model, start_string=u"HAN SOLO: "))

HAN SOLO: I'll rat. heatus,
TiClY REY:
Nipencle, beyks' mose Abouth,y
WHetchookim! HaSZREY:
WySK:
The Sowary give--genu ofe'll live: I'll even-roy, he
neprey'd knoct a
yebqueb.
Whuenden wavagliney kne,
wh sin: Grayi.''e
Greefulp's reclippige.'Tu lio younatoms.,-y uI

ELdBever'tw CAMquazOPRY:
Nap. Jatf-Happy, ersm.
MaRCALETES:
Whyoug cozmanly, 'PePPstrades Romi;
bencedest, goubflip-dy.
sVoeds! fom-sequirx'dry offord.
Homaliva, fity grasw-hidkent 'gpokn flooth
Noken weech!--Gut, Wgerchemoly,
I'll labood abouch! Om,
Sthe ecvess'd.ON&BEEH:
He sloy'd-hence?
Ozs, a
eagenea;
Mysseft! would'st kmoke YORY: tyRINGXSRUCH:
Forget VyVile.
Phevidess'ff.
D no is,.
Gyen inked SacoGO NI
Grovidmatiposeo,
Unomey, so. Mhgides hove hist;
Foll, yourlowarxes serD.' Likes nay!' LaULIUCL:
YOiK:
Tearby cupn'
Mistechby ou getropyou, back nequr.

DUKE!!
Welch bowel; RaQLICIOI:-wa!
Orqum, behof, folse trrehol mysealls'
CyrBYo! 'SquiebroY;--

ALfAMll!!
I'ClxORUD:
Johg-Hadizag; celsher 'CUpo. Colta.
If and effecclis